PredictIt Arbitrage Opportunities
===

I put this sheet together to point out the Arbitrage opportunities on the PredictIt markets. It appears that it is often possible to cross the spread in a number of related contracts and lock in a guarneteed profit and this sheet demonstrates this.

In [1]:
import predictitpy as pp
import itertools

__PredictItPy__

Initializes a predictitpy client which will query the PredictIt market data API.

In [2]:
pc = pp.Client()

__Candidates__

We need a list of candidates for each party.

In [3]:
dem_candidates = ['SANDERS', 'CLINTON']
gop_candidates = ['KASICH', 'CRUZ', 'TRUMP']

__Primaries__

We also need a list of primaries to look at. I have found arbitrate opportunities in some of these primaries, you may want to look at others though.

In [4]:
primaries = [ 'NYPRMRY16', 'PAPRMRY16', 'MDPRMRY16', 'CTPRMRY16', 'RIPRMRY16' ]

__Parties__

We also need a listing of parites so we can easily construct market tickers.

In [5]:
parties = ['GOP', 'DEM']

__Market Tickers__

This produces a list of tickers for each 'market' of related contracts (each primary).

In [6]:
market_tickers = ['%s.%s' % z for z in itertools.product(primaries, parties)]

__Tickers__

This creates a list of all the tickers of all the contracts that we wish to check.

In [7]:
tickers = [ '%s.%s.GOP' % z for z in itertools.product(gop_candidates, primaries)] + \
    [ '%s.%s.DEM' % z for z in itertools.product(dem_candidates, primaries)] 

__Get Data__

This actually gets the data for each ticker.

In [8]:
data = map(pc.get_contract, tickers)

__Finding Arbs__

This function finds arb for a given market_ticker (primary) and the data.

We consider something an arb if we can cross the spread to buy all the contracts in one market (primary) for less that 1. or cross the spread to sell all the contracts in one market (primary) for more than 1.

This function returns a pair (type, profit) where type is one of 'sell', 'buy', 'both' or 'none', where 'buy' means we should buy all the contracts in a market (primary) and so on. profit is the maximum potential profit which can be extracted from the arb.

If we consider tail risks of none of the listed candidates winning the primary, we would prefer 'sell' arbs to 'buy' arbs.

In [9]:
def find_arb(market_ticker, data):
    contracts = [d for d in data if market_ticker in d['TickerSymbol']]
    buys = [b['BestBuyYesCost'] for b in contracts]
    sells = [s['BestSellYesCost'] for s in contracts]
    if sum(buys) < 1. and sum(sells) > 1.:
        return ('both', max(1.-sum(buys), sum(sells)-1.))
    elif sum(buys) < 1.:
        return ('buy', 1.-sum(buys))
    elif sum(sells) > 1.:
        return ('sell', sum(sells) - 1.)
    else:
        return ('none', 0.)

__Find Arbs__

This produces a listing of arbitrage opportunities.

In [11]:
[(mt, find_arb(mt, data)) for mt in market_tickers]

[('NYPRMRY16.GOP', ('none', 0.0)),
 ('NYPRMRY16.DEM', ('none', 0.0)),
 ('PAPRMRY16.GOP', ('sell', 0.030000000000000027)),
 ('PAPRMRY16.DEM', ('none', 0.0)),
 ('MDPRMRY16.GOP', ('none', 0.0)),
 ('MDPRMRY16.DEM', ('none', 0.0)),
 ('CTPRMRY16.GOP', ('none', 0.0)),
 ('CTPRMRY16.DEM', ('sell', 0.050000000000000044)),
 ('RIPRMRY16.GOP', ('sell', 0.020000000000000018)),
 ('RIPRMRY16.DEM', ('none', 0.0))]